# Introduction
This text gives an introduction to the undulator code in this package, with a concentration on the physics/maths implemented within it rather than instructions on how to use the code.  Such instructions will be documented later.

# ElectronBeam
The electron-beam representation in this package is very simple.  The initialisation function takes the energy of the beam, the fractional energy spread, and the emittance and beta-functions at the centre of the unulator.

The class assumes that the beam has a waist at the centre of the undulator, and that the dispersion has been nulled for this section of the accelerator, and so the 4D transverse spot size can be calculated directly.

$$\sigma_{x,y}=\sqrt{\epsilon_{x,y}\beta_{x,y}}$$
$$\sigma'_{x,y}=\sqrt{\frac{\epsilon_{x,y}}{\beta_{x,y}}}$$

This class also knows its relativistic $\gamma_0$ factor, calculated directly from the beam energy (and assuming electrons).

# InsertionDevice
The representation of the insertion device is also very simple.  This class is basically a storage space for the parameters of the device -- $K_{max}$, undulator period ($\lambda_w$), number of periods ($N_{p}$), and the length of the device ($L$).

Note that no checking is done to ensure that $L = N_{p}\lambda_w$.

# Beamline
This is the class containing all of the physics complexity.

## Spectral properties
The wavelength, $\lambda_n$, of a particular harmonic, $n$, when viewed with an angle, $\theta$, with respect to the forward direction is calculated as follows.

$$
\lambda_n = \frac{\lambda_w}{2n\gamma_0^2}\left(1 + K_{max}^2 + \gamma_0^2\theta^2\right)
$$

From this, the photo energy, $E_\gamma$, can be calculated (where $h$, $c$, and $e$, are the usual physical constants).

$$
E_\gamma = \frac{hc}{e}\frac{1}{\lambda_n}
$$

There are two primary contributors to the width of each spectral line: the finite number of undulator periods, and the spread of the beam parameters that appear in that expression (i.e., the electron energy and divergence).

### Beam contribution
The width, $\sigma_{\lambda_{n,b}}$, due to the electron beam parameter spreads is calculated from a Taylor expansion of the $\lambda_n$ expression.

$$\Delta\lambda_n = \frac{\partial\lambda_n}{\partial \gamma} \Bigr|_{\substack{\gamma=\gamma_0}} \cdot \Delta\gamma + \frac{\partial\lambda_n}{\partial\Theta}\Bigr|_{\substack{\Theta=0}}\cdot\Delta\Theta + \frac{1}{2}\frac{\partial^2\lambda_n}{\partial\Theta^2}\Bigr|_{\substack{\Theta=0}}\cdot\Delta\Theta^2
$$

$$
\frac{\partial\lambda_n}{\partial \gamma}\Bigr|_{\substack{\gamma=\gamma_0}} = -\frac{1}{n\gamma_0^3}\left(1+K_w^2\right)\lambda_w
$$

$$
\frac{\partial\lambda_n}{\partial\Theta}\Bigr|_{\substack{\Theta=0}}=\frac{\Theta\lambda_w}{n}\Bigr|_{\substack{\Theta=0}} = 0
$$

$$
\frac{\partial^2\lambda_n}{\partial\Theta^2}\Bigr|_{\substack{\Theta=0}}=\frac{\lambda_w}{n}
$$

To calculate the RMS spread, it is assumed that the distributions of the energy-spread and the angular divergence are uncorrelated, and so can be added in quadrature.

$$
\sigma_{\lambda_{n,b}}^2 = \left(-\frac{1}{n}\frac{1}{\gamma_0^3}\left(1+K_w^2\right)\lambda_w\cdot\Delta\gamma\right)^2 + \left(\frac{1}{2}\frac{1}{n}\lambda_w\cdot\Delta\Theta^2\right)^2
$$

### Undulator contribution
The energy spectrum, $\frac{dW_n}{d\omega}$, of the $n$-th harmonic goes as follows.

$$
\frac{dW_n}{d\omega} \propto \left(\frac{\sin\left(\pi N_p \frac{\Delta\omega_n}{\omega_1}\right)}{\pi N_p \frac{\Delta\omega_n}{\omega_n}}\right)^2
$$

In many texts, the width of the spectrum due to the $\frac{\sin\left(Nx\right)}{Nx}$ is often approximated as the point at which this function first passes through zero.  This solution results in, $\frac{\Delta\omega_n}{\omega_n}=\pm\frac{1}{nN_p}$.  Unfortunately this does not match well with the RMS widths used elsewhere in the code, and so a different point is chosen.

$$
\frac{\sin\left(Nx\right)}{Nx} = \sqrt{\frac{1}{2}} \Longrightarrow \frac{\Delta\omega_n}{\omega_n}=\frac{0.225079}{N_p}
$$

This results in a spectral width, $\sigma_{\lambda_{n,u}}$, due to the finite number of periods in the undulator of,
$$
\sigma_{\lambda_{n,u}} = 0.225079\frac{1}{nN_p}\lambda_n
$$

### Total spectral width
The beam contribution and the undulator contribution should be convoluted to optain the total width, however it is enough to add the contributions in quadrature.

$$
\sigma_{\lambda_{n,total}}^2 = \left(-\frac{1}{n}\frac{1}{\gamma_0^3}\left(1+K_w^2\right)\lambda_w\cdot\Delta\gamma\right)^2 + \left(\frac{1}{2}\frac{1}{n}\lambda_w\cdot\Delta\Theta^2\right)^2 + \left(0.225079\frac{1}{nN_p}\lambda_n\right)^2
$$

## Example of a calculation of spectral properties
First import the necessary components.

In [2]:
import undulator as phys

In [7]:
Keff = 1.38
lamda_w = 18e-3
E = 3e9

ebeam = {
    'energy': E,
    'emitx': 340e-12,
    'emity': 8e-12,
    'betax': 9,
    'betay': 4.7,
    'espread': 0.8e-3,
}
print(phys.sigy(ebeam)*1e6)
print(phys.sigyp(ebeam)*1e6)
print(phys.sigx(ebeam)*1e6)
print(phys.sigxp(ebeam)*1e6)

6.131883886702356
1.3046561461068842
55.31726674375732
6.146362971528592


In [6]:
Keff = 1.38
lamda_w = 18e-3
E = 3e9

achr3 = InsertionDevice(Kmax=Keff, lamda_w=lamda_w, Np=111, L=2)
r3 = ElectronBeam(energy=E,
                  emittance={'x': 340e-12, 'y': 8e-12},
                  beta={'x': 9, 'y': 4.7},
                  espread=0.8e-3
                 )
ins_dev = {
    'Kmax': Keff,
    'lamda_w': lamda_w,
    'Np': 111,
    'L': 2,
}
ebeam = {
    'energy': E,
    'emittance': {'x': 340e-12, 'y': 8e-12},
    'beta': {'x': 9, 'y': 4.7},
    'espread': 0.8e-3,
}
nanomax = Beamline(undulator=achr3, electronbeam=r3)

n=5
print(nanomax)
print(f'Brightness => {nanomax.brightness(n)}')
print(f'Max brightness => {nanomax.brightness_max(n)}')
print(f'Ratio => {nanomax.brightness_max(n) / nanomax.brightness(n)}')

InsertionDevice:: Kmax=1.38 Period=18.0 mm Ncells=111
ElectronBeam:: 3.0 GeV ex=3.4e-10 ey=8e-12 bx=9 by=4.7
Brightness => 7.601790863114727e+22
Max brightness => 1.6917987725241892e+25
Ratio => 222.5526593651905
